In [21]:
import ipywidgets as widgets
import traitlets

Consider this super-simple (and super-bad) password generator widget: given a password length, it constructs a sequence of random letters of that length and displays it. In this notebook we'll walk through two ways of constructing that widget and introduce the concept oif using traitlets to connect the password calculation code to the widget's graphical interface.

## Construct the interface (widget)

This time we will build the password widget as a subclass of `VBox`. This has the virtue of making it easy to distribute and use just like one of `ipywidget`'s built in widgets.

In [15]:
class PassGenGUI_v1(widgets.VBox):
    def __init__(self):
        # Do NOT forgot to do this!
        super(PassGenGUI_v1, self).__init__()
        
        # Define each of the children...
        self._helpful_title = widgets.HTML('Generated password is:')
        self._password_text = widgets.HTML('No password yet', placeholder='No password generated yet')
        self._password_text.layout.margin = '0 0 0 20px'
        self._password_length = widgets.IntSlider(description='Length of password',
                                                  min=8, max=20,
                                                  style={'description_width': 'initial'})
        children = [self._helpful_title, self._password_text, self._password_length]
        self.children = children

In [16]:
pwd_gen = PassGenGUI_v1()
pwd_gen

A Jupyter Widget

In [22]:
pwd_gen.add_traits(value=traitlets.Unicode())

Notice that our widget doesn't have a value; it really seems like it should, and that the value should be the password itself. Though we could add that as a `traitlet` linked to the value of the `_password_text`, we'll add a read-only property to the class instead. The reason is that we don't want users to be able to set the value of our widget's password. Its entire purpose is to generate a passsword for the user.

In [37]:
class PassGenGUI_v2(widgets.VBox):
    def __init__(self):
        # Do NOT forgot to do this!
        super(PassGenGUI_v2, self).__init__()
        
        # Define each of the children...
        self._helpful_title = widgets.HTML('Generated password is:')
        self._password_text = widgets.HTML('No password yet', placeholder='No password generated yet')
        self._password_text.layout.margin = '0 0 0 20px'
        self._password_length = widgets.IntSlider(description='Length of password',
                                                  min=8, max=20,
                                                  style={'description_width': 'initial'})
        children = [self._helpful_title, self._password_text, self._password_length]
        self.children = children
        
    @property
    def value(self):
        return self._password_text.value

In [38]:
pwd_gen2 = PassGenGUI_v2()
pwd_gen2

A Jupyter Widget

In [39]:
pwd_gen2.value

'No password yet'

In [41]:
pwd_gen2.value = 'new password'

AttributeError: can't set attribute

## An alternative to callbacks: a model with traitlets, linked to the GUI

In the previous iteration of our password generator we wrote a function to generate a password, and a second function to add a callback, i.e. a call to that function whenever a control changes. There is nothing wrong with that approach, but it gets more complicated the more controls you add.

Imagine we add a second control that allows the user to choose whether to include numbers in the password. In addition to modifying the password generating function, we would either need to write a function for handling changes and make the new numerical checkbox `observe` the new function, or modify `update_password` to check which widget generated the event and make the apropriate call to `calculate_password`.

Instead, we define a class below that generates the password, and includes as class attributes `traitlets` that we will later link to the widget GUI. Note that nothing in this class refers to the widget; it's logic is entirely internal.

To include traitlets a class must subclass from `traitlets.HasTraits`.

In [52]:
# Subclass from HasTraits
class PassGen_v1(traitlets.HasTraits):
    # Define your traits here, as class attributes 
    length = traitlets.Integer()
    
    # initialize the class by calling the superclass
    def __init__(self):
        super(PassGen_v1, self).__init__()
        
    # The observe decorator is used to indicate that the function being
    # decorated should be called if any of the traits listed as arguments
    # change. The decorated function must have an argument named change;
    # in this example the change variable isn't used because we want to update
    # the generated password no matter what the change was.
    
    # Note that we are monitoring changes in PassGen's own trait, length,
    # rather than a change in one of the controls in the GUI.
    
    @traitlets.observe('length')
    def calculate_password(self, change):
        import string
        import random
    
        # Generate a list of random letters of the correct length.
        new_password = [random.choice(string.ascii_letters) for _ in range(self.length)]
        new_password = ''.join(new_password)
        
        # We don't return anything (explanation later). For now just print the 
        # password to observe what happens when we change the length.
        print('In password generator password is: ', new_password)

In [53]:
p = PassGen_v1()

Try changing the length property of in the cell below. Note that any time it is changed, a new password is generated.

In [54]:
p.length = 100

In password generator password is:  MOePeSHhbcewOxckeKNKEoEMIGjCfXABwPwvQemHChCCeUtiwhiPBWSzuYLyKqdrUbkOMgKdNyqXEeWlhMBYNIDnZjewPXQHlYHu


We don't actually want to print the password out each time we generate a new one, of course, and we eventually want to connect this to a widget-based GUI. The class below adds a new trait for the generated password, and sets that trait in `calculate_password`. You could, in addition, return the generated password; that could be useful in testing your model code, for example.

In [59]:
# Subclass from HasTraits
class PassGen_v2(traitlets.HasTraits):
    # Define your traits here, as class attributes 
    length = traitlets.Integer()
    
    # The new trait is defined here:
    password = traitlets.Unicode()
    
    def __init__(self):
        super(PassGen_v2, self).__init__()
            
    @traitlets.observe('length')
    def calculate_password(self, change):
        import string
        import random
    
        new_password = [random.choice(string.ascii_letters) for _ in range(self.length)]
        new_password = ''.join(new_password)
        
        # Set the value of the password trait here:
        self.password = new_password

Now let's try it out:

In [60]:
p2 = PassGen_v2()

In [69]:
p2.length = 11
p2.password

'VMWfVwkTYtQ'

Now that we have an interface and a model to encapsulate the calcualtion we want to do, we need to connect them. We will do it by creating the links in the GUI class. The code below is identical to `PassGenGUI_v2` from above, with the new lines explained in comments.

In [81]:
class PassGenGUI_v3(widgets.VBox):
    def __init__(self):
        super(PassGenGUI_v3, self).__init__()
        
        self._helpful_title = widgets.HTML('Generated password is:')
        self._password_text = widgets.HTML('', placeholder='No password generated yet')
        self._password_text.layout.margin = '0 0 0 20px'
        self._password_length = widgets.IntSlider(description='Length of password',
                                                  min=8, max=20,
                                                  style={'description_width': 'initial'})
        children = [self._helpful_title, self._password_text, self._password_length]
        self.children = children
        
        # Create an instance of our model.
        self.model = PassGen_v3()
        
        # link the password from the model to the _password_text widget
        traitlets.link((self.model, 'password'), (self._password_text, 'value'))
        
        # link the _password_length widget to the length in the model
        traitlets.link((self.model, 'length'), (self._password_length, 'value'))
        
    @property
    def value(self):
        return self._password_text.value

Let's create an instance of the class, display it, and try it out!

In [82]:
password_widget = PassGenGUI_v3()
password_widget

A Jupyter Widget

Notice that as currently written nothing prevents setting a nonsensical length:

In [70]:
p2.length = -14
p2.password

''

Traitlets provides a mechanism for validating values using the `@validate` decorator. The documentation on validation is [here](https://traitlets.readthedocs.io/en/stable/using_traitlets.html#validation), but the essential ideas are illustrated below.

In [75]:
# Subclass from HasTraits
class PassGen_v3(traitlets.HasTraits):
    # Define your traits here, as class attributes 
    length = traitlets.Integer()
    password = traitlets.Unicode()
    
    def __init__(self):
        super(PassGen_v3, self).__init__()
            
    @traitlets.observe('length')
    def calculate_password(self, change):
        import string
        import random
    
        new_password = [random.choice(string.ascii_letters) for _ in range(self.length)]
        new_password = ''.join(new_password)
        
        # Set the value of the password trait here:
        self.password = new_password
    
    # The new validator:
    @traitlets.validate('length')
    def _moo(self, proposal):
        length = proposal['value']
        if length < 1:
            raise traitlets.TraitError('Password length should be positive.')
        return proposal['value']

In [76]:
p3 = PassGen_v3()

In [77]:
p3.length = -23

TraitError: Password length should be positive.

In [78]:
p3.length = 25
p3.password

'XVsChEKxKpfLwhzfzzUiGKYjt'